#### CLEAN DATASETS FROM CARTO-SONUM

10/03/2019 - Julien Paris

for each dataset : 
- read CSV/EXCEL/JSON
- get unique values from all columns
- retrieve unique values on specific columns
- export original data as CSV and XLS files
- export unique values as a JSON file

cf : https://github.com/co-demos/carto-sonum/tree/master/data

In [1]:
# -*- encoding: utf-8 -*-
import json
import ast

import pandas as pd
from pandas.io.json import json_normalize
from pandas.api.types import is_string_dtype
import numpy as np

In [2]:
CSVfiles = [ 

    ### DRJSCS
    {"name" : "export_base_ressources_territoriales_20181227_DRJSCS_HDF", 
     "ext" : "csv",
     "encoding" : "ISO-8859-1",
     "sep" : ";",
     "colsToExclude" : [],
     "colToGetUniques" : [
         u"Enjeu", 
         u"Dispositif", 
         u"Public_cible", 
         u"Statut",
         u"Echelle",
         u"Conditions_acces",
         u"Horaires"
     ],
     "code" : "DRJSCS"
    },
    
    ### MSAP
    {"name" : "MSAP-20180627", 
     "ext" : "csv",
     "encoding" : "ISO-8859-1",
     "sep" : ";",
     "colsToExclude" : [],
     "colToGetUniques" : [
         u"Horaires d'ouverture",
         u"Accessibilité",
         u"Equipements à disposition"
     ],
     "code" : "MSAP"
    },
    
    ### LOIRE ATLANTIQUE
    {"name" : "224400028_lieux-numeriques-en-loire-atlantique", 
     "ext" : "csv",
     "encoding" : "utf-8",
     "sep" : ";",
     "colsToExclude" : [],
     "colToGetUniques" : [
         u"Type de porteur", 
         u"Public",
         u"Tarif(s)"
     ],
     "code" : "Loire-Atlantique"
    },
    
    ### GIRONDE
    {"name" : "listeEPN-NETPUBLIC-Gironde", 
     "ext" : "csv",
     "encoding" : "utf-8",
     "sep" : ",",
     "colsToExclude" : [],
     "colToGetUniques" : [
         u"Label",
         u"Services proposés",
         u"Tarif"
     ],
     "code" : "Gironde"
    },
    
    ### NET PUBLIC 
    {"name" : "Copie de Annuaire EPN Netpublic - decembre 2016", 
     "ext" : "csv",
     "encoding" : "ISO-8859-1",
     "sep" : ";",
     "colsToExclude" : [],
     "colToGetUniques" : [
         u"Statut",
     ],
     "code" : "NetPublic"
    },
    
    ### APTIC
    {"name" : "APTIC - extractCatalog", 
     "ext" : "json",
     "encoding" : "utf-8",
     "sep" : "",
     #"list_fields" : ["services"],
     "colsToExclude" : ["nom_representant", "prenom_representant"],
     "colToGetUniques" : [
         u"Statut",
     ],
     "code" : "APTIC"
    },
]

In [11]:
class FiletoXLS :
    
    def __init__(self, name, code, ext="csv",sep=";", encoding="utf-8", list_fields=[], cols_to_exclude=[]) : 
        self.filename = name
        self.code = code
        self.ext = ext
        self.file = name + "." + ext
        self.sep = sep
        self.list_fields = list_fields
        self.cols_to_exclude = cols_to_exclude
        self.encoding = encoding
        self.df = None
        self.columnsNames = []
        self.columnsUniques = []

    def toDF(self):
        if self.ext == "csv":
            df = pd.read_csv(self.file, sep=self.sep, encoding=self.encoding)
        elif self.ext == "json":
            with open(self.file) as f:
                data = json.load(f)
            ### normalize columns names
            df = json_normalize(data)
            ### convert lists to strings 
            #df = df.to_string()
        else : 
            df = pd.read_excel(self.file, encoding=self.encoding)
        print df.head(3)
        df = df.dropna(how="all")
        df = df.replace({np.nan:None}) 
        self.df = df[ [c for c in df.columns if c not in self.cols_to_exclude] ]
        return df
    
    def getUniques(self):
        results=[]
        for col in self.df:
            print "---- \ncol : %s" %(col)
            print "\n col[:2] : %s" %(col[:2])

            self.columnsNames.append(col)
            
            ### check serie type
            #is_col_str_type = is_string_dtype(self.df[col])
            #print "\nis_col_str_type : %s" %(is_col_str_type)
            col_dtype_kind = self.df[col].dtype.kind
            print "\n col_dtype_kind : %s" %(col_dtype_kind)

            if col in self.list_fields: 
                self.df[col] = self.df[col].apply(lambda x: ' / '.join(x))
                
            uniques = self.df[col].unique()
            u_dict = {"colname" : col, "uniques" : uniques.tolist() }
            u_dict["u_count"] = len(u_dict["uniques"])
            results.append(u_dict)
        self.columnsUniques = results
        return results
            
    def toXLS(self):
        #writer = pd.ExcelWriter(self.code + '.xlsx')
        #self.df.to_excel(writer, index = False)
        #writer.save()
        filename = "-" + self.code + '.xlsx'
        self.df.to_excel(filename, encoding="utf-8")
    
    def toTSV(self):
        filename = "-" + self.code + '.csv'
        self.df.to_csv(filename, sep='\t', encoding='utf-8')
        
    def toJSON(self):
        filename = "-" + self.code + '.json'
        self.df.to_json(filename, orient="records")


In [17]:
#CSVlist = [CSVfiles[1]]
CSVlist = CSVfiles

In [18]:

CSV_dfList = []
for csv in CSVlist:
    list_fields = csv.get("list_fields", [])
    classCSV = FiletoXLS(
        csv["name"],csv["code"],
        ext=csv["ext"],sep=csv["sep"],
        encoding=csv["encoding"], 
        list_fields=list_fields,
        cols_to_exclude=csv["colsToExclude"],
    )
    classCSV.toDF()
    classCSV.getUniques()
    result = { 
        "code" : csv["code"], 
        "df" : classCSV.df, 
        "columnsNames" : classCSV.columnsNames,
        "columnsUniques" : classCSV.columnsUniques,
        "classCSV" : classCSV, 
        "count" : {"rows" : classCSV.df.shape[0],"columns" : classCSV.df.shape[1] },
        "csvInfos" : csv,
        "colUniquesToKeep" : [ c for c in classCSV.columnsUniques if c["colname"] in csv["colToGetUniques"]]
    }
    CSV_dfList.append(result)

    Id          Enjeu                                         Dispositif  \
0  617  Accessibilit  Mise  disposition de matriel informatique et...   
1  616  Accessibilit  Mise  disposition de matriel informatique et...   
2  590  Accessibilit  Mise  disposition de matriel informatique et...   

  Public_cible                              Descriptif  \
0  Tout public  Accs  Internet dans la bibliothque.   
1  Tout public  Accs  Internet dans la bibliothque.   
2  Tout public  Accs  Internet dans la bibliothque.   

         Structure_porteuse              Type_structure  Statut  \
0       Bibliotheque Rurale  Bibliothque - Mdiathque  Public   
1  Mediatheque Henry Murger  Bibliothque - Mdiathque  Public   
2   Bibliotheque Municipale  Bibliothque - Mdiathque  Public   

                           Adresse           Commune      ...         Dpt  \
0       PLACE DU GENERAL DE GAULLE            Pavant      ...       Aisne   
1               PARC DU 8 MAI 1945   No


 col[:2] : De

 col_dtype_kind : O
---- 
col : Logo

 col[:2] : Lo

 col_dtype_kind : O
---- 
col : Photo

 col[:2] : Ph

 col_dtype_kind : O
---- 
col : Public

 col[:2] : Pu

 col_dtype_kind : O
---- 
col : Accès

 col[:2] : Ac

 col_dtype_kind : O
---- 
col : Restrictions d'usage

 col[:2] : Re

 col_dtype_kind : O
---- 
col : Tarif(s)

 col[:2] : Ta

 col_dtype_kind : O
---- 
col : Horaires du lundi

 col[:2] : Ho

 col_dtype_kind : O
---- 
col : Horaires du mardi

 col[:2] : Ho

 col_dtype_kind : O
---- 
col : Horaires du mercredi

 col[:2] : Ho

 col_dtype_kind : O
---- 
col : Horaires du jeudi

 col[:2] : Ho

 col_dtype_kind : O
---- 
col : Horaires du vendredi

 col[:2] : Ho

 col_dtype_kind : O
---- 
col : Horaires du samedi

 col[:2] : Ho

 col_dtype_kind : O
---- 
col : Horaires du dimanche

 col[:2] : Ho

 col_dtype_kind : O
---- 
col : Complément sur les horaires

 col[:2] : Co

 col_dtype_kind : O
---- 
col : Ordinateur(s)

 col[:2] : Or

 col_dtype_kind : i
---- 
col : 

/Users/jpy/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (33,58,59,63,66,68,69,70,71,72,75,77,78,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,99,101,102,103,104,105,106,108,110,111,113,114,115,116,117,119,120,121,122,123,124,125,126,127,128,129,130,131) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


                Identifiant Date de dernire modification  \
0  54ec6c76de77fe0314ee3a1d                    24/02/2015   
1  4f340209de77fe55ff000e7b                    14/02/2012   
2  4f340209de77fe55ff000e7a                    14/02/2012   

                               Nom                 Statut  \
0              Association Lalouma  Structure associative   
1  Espace Cyber-base EPM de Lavaur                    NaN   
2                         APER-CTL                    NaN   

                 Labels territoriaux  Label NetPublic  \
0        PassNumrique (Rhne Alpes)                1   
1                                NaN                1   
2  Cyber-centre (Nord-Pas-de-Calais)                1   

   Mise en oeuvre du dispositif "2000 Emplois d'Avenir en EPN"  \
0                                                0.0             
1                                                NaN             
2                                                NaN             

   Demande de la


 col[:2] : T

 col_dtype_kind : O
---- 
col : Fax

 col[:2] : Fa

 col_dtype_kind : O
---- 
col : Intitul

 col[:2] : In

 col_dtype_kind : O
---- 
col : Commentaire

 col[:2] : Co

 col_dtype_kind : O
---- 
col : Autre Site Web

 col[:2] : Au

 col_dtype_kind : O
---- 
col : Personne  contacter.1

 col[:2] : Pe

 col_dtype_kind : O
---- 
col : Offre de services spcifiques

 col[:2] : Of

 col_dtype_kind : O
---- 
col : N¡ et libell de voie.4

 col[:2] : N¡

 col_dtype_kind : O
---- 
col : N¡ et libell de voie.5

 col[:2] : N¡

 col_dtype_kind : O
---- 
col : Code Insee commune.2

 col[:2] : Co

 col_dtype_kind : O
---- 
col : Code postal.2

 col[:2] : Co

 col_dtype_kind : O
---- 
col : Localit.2

 col[:2] : Lo

 col_dtype_kind : O
---- 
col : N¡ et libell de voie.6

 col[:2] : N¡

 col_dtype_kind : O
---- 
col : Autre site web

 col[:2] : Au

 col_dtype_kind : O
---- 
col : Site Web2

 col[:2] : Si

 col_dtype_kind : O
---- 
col : Vido

 col[:2] : Vi

 col_dtype_kind : O
--

In [19]:
for csv in CSV_dfList:
    print "- code : %s" %(csv["code"])
    print "- count : %s" %(csv["count"])
    print "- columnsNames : " 
    #print csv["columnsNames"]
    for c in csv["columnsNames"] : 
        print c
    print
    col = csv["columnsUniques"]
    for c in col : 
        if c in csv["colUniquesToKeep"]: 
            print "- colname : %s" %(c["colname"])
            print "- uniques[0:3] : %s..." %(c["uniques"][0:3])
            print "- u_count : %s" %(c["u_count"])
            print "- "*5
    print "=== "*10

- code : DRJSCS
- count : {'rows': 1404, 'columns': 21}
- columnsNames : 
Id
Enjeu
Dispositif
Public_cible
Descriptif
Structure_porteuse
Type_structure
Statut
Adresse
Commune
CP
Dpt
Region
Echelle
Courriel
Telephone
Accompagnement
Conditions_acces
Handicaps
Horaires
Site_internet

- colname : Enjeu
- uniques[0:3] : [u'Accessibilit\x8e', u'Formation et Accompagnement', u'Simplification']...
- u_count : 3
- - - - - 
- colname : Dispositif
- uniques[0:3] : [u"Mise \x88 disposition de mat\x8eriel informatique et d'une connexion internet en acc\x8fs libre", u"Point d'accueil num\x8erique", u'Accompagnement aux d\x8emarches administratives - Accueil Caf\x8e itin\x8erant']...
- u_count : 145
- - - - - 
- colname : Public_cible
- uniques[0:3] : [u'Tout public', u"Demandeurs d'emploi", u'S\x8eniors \x8eloign\x8es du num\x8eriques']...
- u_count : 88
- - - - - 
- colname : Statut
- uniques[0:3] : [u'Public', u'Priv\x8e', u'Public / Priv\x8e']...
- u_count : 6
- - - - - 
- colname : Echelle
- uni

In [20]:
#CSV_dfList[0]["df"].head(3)

In [21]:
for csvClass in CSV_dfList:
    csvClass["classCSV"].toXLS()
    csvClass["classCSV"].toTSV()
    csvClass["classCSV"].toJSON()

    ### dump json only for uniques
    with open( "-" + csvClass["code"] + '-infos.json', 'w') as fp:
        dict_to_dump = {
            k:v for k,v in csvClass.iteritems() if k in [
                "code", "columnsNames", "count", "csvInfos", "colUniquesToKeep"
            ]
        }
        json.dump(dict_to_dump, fp)

/Users/jpy/anaconda2/lib/python2.7/site-packages/xlsxwriter/worksheet.py:830: UserWarning: Ignoring URL 'http://mediatheque.mairie-verrieres-91.fr/
Accs%20%20l'espace%20multimdia%20libre%20sur%20rservation%20sur%20place%20ou%20par%20tlphone

Mardi%20:%2016h00%20-%2019h00
Mercredi%20:%2010h00%20-%2012h30%20%20//%2014h00%20-%2018h00
Jeudi%20:%20Fermeture
Vendredi%20:%20Fermeture
Samedi%20:%2010h00%20-%2012h30%20//%2014h00%20-%2018h00' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/Users/jpy/anaconda2/lib/python2.7/site-packages/xlsxwriter/worksheet.py:830: UserWarning: Ignoring URL 'http://mobile18.fr/du-materiel-innovant-9

Les%20animateurs%20du%20Mobile%20se%20dplacent%20dans%20un%20vhicule%20facilement%20identifiable%20!%20Mais%20attention,%20contrairement%20%20certaines%20ides%20reues,%20ce%20ne%20sont%20pas%20les%20participants%20qui%20montent%20dans%20le%20vhicule%20mais%20bien%20le%20matriel%20qui%20en%20so